# Wolf Sighting Prediction - Training and Prediction

### Import Libraries

All the necessary libraries are imported here. They are listed in `requirements.txt` and can be installed using the following command:

```bash
pip install -r requirements.txt
```

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, roc_auc_score
import joblib
from ipyleaflet import Map, Marker
from ipywidgets import DatePicker, VBox, Button, Output
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve

from common import *

### Data Loading

We load the training data consisting of wolf sighting events from the database repository. Additionally, we load historical sighting data to infer recent sightings in the region when running the prediction for a given location and date. To determine region IDs, we also need to load our KMeans model.

In [ ]:
# Download training and testing data from the database repository
events_download = DownloadDataWidget(
    use_auth=False,
    text="Load Train Events",
    database_id='4c9ac630-7ec5-491c-b727-0bea3224da91',
    table_id='185815ae-df08-4fc3-aacc-ddd886ab017b',
)
events_download.display()
events_download.load_data()

# Download sighting history from the database repository
sightings_download = DownloadDataWidget(
    use_auth=False,
    text="Load Sightings",
    database_id='4c9ac630-7ec5-491c-b727-0bea3224da91',
    table_id='86151a41-a870-4799-9441-aaef7a4d23cc',
)
sightings_download.display()
sightings_download.load_data()

# Load the KMeans model
# todo: should be loaded from TUWRD
kmeans_model = joblib.load('data/wolf_sightings_kmeans.pkl')

## Model Training

For the machine learning model, a Random Forest Classifier is used. Numeric features are standardized and categorical features are one-hot encoded.

In [ ]:

classifier = RandomForestClassifier(random_state=42, n_estimators=100)

# Define preprocessing
categorical_features = [
    'season',
    'coord_bin',
    'region_id',
]
numeric_features = [
    'month',
    'lat', 'lon',
    'recent_sightings',
    'days_since_last_sighting',
]

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
], remainder='passthrough')

# Define model pipeline
pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('clf', classifier)
])

Before training, the target variable (`is_sighting`) is separated from the features, and the data split into training and testing sets. The model is then trained using the training data. Afterwards, it is evaluated on the test set and the accuracy is printed.

In [ ]:
training_data = events_download.get_data().copy()

# Ensure right data types
training_data['lat'] = training_data['lat'].astype(float)
training_data['lon'] = training_data['lon'].astype(float)
training_data['month'] = training_data['month'].astype(int)
training_data['region_id'] = training_data['region_id'].astype(int)
training_data['recent_sightings'] = training_data['recent_sightings'].astype(int)
training_data['days_since_last_sighting'] = training_data['days_since_last_sighting'].astype(int)
training_data['season'] = training_data['season'].astype('string')
training_data['coord_bin'] = training_data['coord_bin'].astype('string')
training_data['is_sighting'] = training_data['is_sighting'].map(lambda x: 1 if x == 'true' else 0).astype(int)
training_data.drop(columns=['id'], inplace=True)

# Define features and target
X = training_data.drop(columns=['is_sighting'])
y = training_data['is_sighting']

# Split data
split_index = int(len(X) * train_test_split_ratio)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]
print(f"Training data size: {X_train.shape[0]}, Test data size: {X_test.shape[0]} (Split: {train_test_split_ratio}%). Total size: {X.shape[0]}")

# Train
pipeline.fit(X_train, y_train)

# Evaluate
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))

The trained model can then be saved to the `data` directory.

In [ ]:
# Save the model
joblib.dump(pipeline, model_path)
print(f"Model saved to {model_path}")

## Evaluation

We perform visual evaluation of the model by plotting a confusion matrix, ROC curve, and precision-recall curve. Addionally, we display a bar plot of the ten most important features.

In [ ]:
plot_dir = 'outputs'

# 1. Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig(f"{plot_dir}/confusion_matrix.png")
plt.show()

# 2. ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc_score(y_test, y_proba):.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.grid()
plt.savefig(f"{plot_dir}/roc_curve.png")
plt.show()

# 3. Precision-Recall Curve
precision, recall, thresholds_pr = precision_recall_curve(y_test, y_proba)
plt.figure(figsize=(6, 4))
plt.plot(recall, precision)
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.grid()
plt.savefig(f"{plot_dir}/precision_recall_curve.png")
plt.show()

# 4. Feature Importances
feature_names = numeric_features + list(pipeline.named_steps['preprocess'].named_transformers_['cat'].get_feature_names_out(categorical_features))
importances = pipeline.named_steps['clf'].feature_importances_
feat_imp = sorted(zip(importances, feature_names), reverse=True)[:10]
plt.figure(figsize=(8, 4))
sns.barplot(x=[imp for imp, name in feat_imp], y=[name for imp, name in feat_imp])
plt.title('Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.savefig(f"{plot_dir}/feature_importances.png")
plt.show()

## Interactive Model Testing

To test the model, an interactive interface is provided where a date and location can be given. The model will then predict the probability of a wolf sighting in that location on that date. The prediction is based on the trained model and the historical data loaded earlier. The KMeans model is used to determine the region ID for the given location, which is used to filter the historical data for recent sightings in that region. 

In [ ]:
class WolfSightingPredictor:
    def __init__(self, history, model, kmeans):
        self.model = model
        self.kmeans = kmeans
        self.sighting_history = history
        # Ensure timestamps are in datetime format
        self.sighting_history['timestamp'] = pd.to_datetime(self.sighting_history['timestamp'])
        self.cutoff_date = self.sighting_history['timestamp'].max()

    def predict(self, lat, lon, date):
        error = ""
        if date > self.cutoff_date:
            error += f"Selected date is beyond the known cutoff date ({self.cutoff_date.date()}). "
        if lat < area_bounds['lat_min'] or lat > area_bounds['lat_max'] or lon < area_bounds['lon_min'] or lon > area_bounds['lon_max']:
            error += "Selected coordinates are out of bounds."
        region_id = self.kmeans.predict([[lat, lon]])[0]
        sample = pd.DataFrame([{
            'month': date.month,
            'season': season_from_month(date.month),
            'region_id': region_id,
            'coord_bin': coordinate_bin(lat, lon),
            'lat': lat,
            'lon': lon,
            'recent_sightings': count_sightings_in_region(self.sighting_history, region_id, date - recent_duration, date),
            'days_since_last_sighting': count_days_since_last_sighting(self.sighting_history, region_id, date),
        }])
        # Predict
        probability = self.model.predict_proba(sample)[0][1]
        return probability, sample, error

wolf_predictor = WolfSightingPredictor(sightings_download.get_data(), pipeline, kmeans_model)

In [ ]:
class WolfPredictorDemo:
    def __init__(self, predictor, map_center, map_zoom=8):
        self.predictor = predictor
        # Map
        self.map = Map(center=map_center, zoom=map_zoom)
        self.marker = Marker(location=map_center, draggable=True)
        self.map.add_layer(self.marker)
        # Date picker widget
        self.date_picker = DatePicker(
            description='Date',
            value=self.predictor.cutoff_date
        )
        # Output display
        self.out = Output()
        # Predict button
        self.btn = Button(description="Predict")
        self.btn.on_click(self.predict_on_click)

    # Prediction callback
    def predict_on_click(self, b=None):
        lat, lon = self.marker.location
        date = pd.to_datetime(self.date_picker.value)
        if date is None:
            with self.out:
                self.out.clear_output()
                print("Please select a valid date.")
            return
        probability, sample, error = self.predictor.predict(lat, lon, date)
        with self.out:
            self.out.clear_output()
            print(f"Prediction for ({lat:.3f}, {lon:.3f}) on {date.date()}:")
            print(f"Recent sightings in region: {sample['recent_sightings'].values[0]}")
            print(f"Days since last sighting: {sample['days_since_last_sighting'].values[0]}")
            print(f"🧠 Likelihood of sighting: {probability:.2f}")
            if error:
                print("⚠️ " + error)

    def display(self):
        return VBox([self.map, self.date_picker, self.btn, self.out])

# Create and display the predictor demo
wolf_predictor_demo = WolfPredictorDemo(
    wolf_predictor,
    map_center=(
        (area_bounds['lat_min'] + area_bounds['lat_max']) / 2,
        (area_bounds['lon_min'] + area_bounds['lon_max']) / 2
    ),
    map_zoom=8
)
wolf_predictor_demo.display()